# Data Exploration

### T10-4
As a USER, I want to have a Data Exploration Section in the notebook so that I can easily locate the upload and playback functions in the notebook.

# Inference

### T10-7
As a USER, I want to have an Inference Section in the notebook so that I can easily locate the Inference Section in the notebook.

In [3]:
import cv2

def output(video):
    cap = cv2.VideoCapture(video)
    # Check if camera opened
    if not cap.isOpened():
        print("Error opening video file")

    # Get video height and width
    # frame_width = int(cap.get(3))
    # frame_height = int(cap.get(4))

    # Define output file name and format, codec used, define fps and the frame size
    # output_video = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30.0,
    #                                (frame_width,frame_height))

    try:
        while cap.isOpened():
            # Capture frame by frame
            ret, frame = cap.read()

            # Check if there are frames captured
            if not ret:
                # Destroy frame window
                cv2.destroyAllWindows()
                # Release the Video Device
                cap.release()
                break

            # writing the new frame in output
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # pyplot.axis('off')
            # pyplot.imshow(frame)
            # pyplot.title('output')
            # pyplot.show()
            # clear_output(wait=True)

            # Write camera capture into video
            # output_video.write(frame)

            # Display video in a frame
            cv2.imshow('Frame', frame)

            # Press Q to quit player
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                
        # Release the Video Device
        cv2.destroyAllWindows()
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")

    # To catch when stopping cell run in notebook
    except KeyboardInterrupt:
        cv2.destroyAllWindows()
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")

if __name__ == "__main__":
    output("") # video path and file here

Released Video Resource


# Training

### T10-6
As a USER, I want to have a Training Section in the notebook so that I can easily locate the training related functions in the notebook.